# Compare analyzers on Social Physical brain

This notebook shows how saliency maps are computed for social and physical pain brain dataset. We will first train a model and then apply LRP methods. 

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter('ignore')

In [2]:
%matplotlib inline  

import h5py
import imp
import numpy as np
import os

import keras
import keras.backend
import keras.models

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import regularizers
from keras import backend as K

import sys
sys.path.insert(0, '../../')

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis

# Use utility libraries to focus on relevant iNNvestigate routines.
eutils = imp.load_source("utils", "../utils.py")
mnistutils = imp.load_source("utils_mnist", "../utils_mnist.py")





Using TensorFlow backend.


## Data
Load the dataset and keep some images from the test set for the analysis.

In [3]:
def generate_sequences(batch_size, tr_data):
   
    while True:
        """
        #this is working everytime model trained - making Batch-size data
        """
        tr_sample_count = tr_data['X_data'].shape[0]
        tr_sample_idxs = range(0,tr_sample_count)

        # Batch_count 

        batch_count = tr_sample_count / batch_size
        if tr_sample_count % batch_size:
            batch_count = batch_count+1

        # Yield X,y(batch_size), every for loop 
        for i in range(0, int(batch_count)):
            if i == batch_count-1:
                batch_size_idxs = tr_sample_idxs[i*batch_size : ]
            else : 
                batch_size_idxs = tr_sample_idxs[i*batch_size : (i+1)*batch_size]

            batch_size_idxs = sorted(batch_size_idxs)
            X = tr_data['X_data'][batch_size_idxs].reshape(len(batch_size_idxs),68,95,79)
            y = tr_data['y_data'][batch_size_idxs]

            yield X,y

In [4]:
mini_batch_size = 32
data_file_name = '180404_social_physical_masked_cross.hdf5'
label_to_class_name = [str(i) for i in range(2)]

## Model
The next part trains and evaluates a CNN.

In [5]:
def make_custom_model_cnn_2D():
       

    model = Sequential() #0
    model.add(Conv2D(8, (3,3), kernel_initializer='he_normal', padding='same', input_shape=(68,95,79)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Conv2D(16, (3,3), kernel_initializer='he_normal', padding='same'))#4
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3,3), kernel_initializer='he_normal', padding='same'))#7
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), kernel_initializer='he_normal', padding='same')) #10
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Flatten()) #13
    model.add(Dense(128, kernel_initializer='he_normal'))
    model.add(Activation('relu'))

    model.add(Dense(1, activation='softmax', kernel_initializer='he_normal'))
    
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model


In [6]:
# Create & train model
def train_model():

    """
    train model with data in every cross_i. Everytime model trained, get_result and generate_sequences are called
    """
    # Option : when you don't have enough GPUs
#     with tf.device('/cpu:0'):

    with h5py.File(data_file_name, "r") as data:

        for i in range(0,1):  # i is the cross number 59

            print('this is ith iter : ' , i)

            tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_train'
            tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_train'

            tr_data = {}
            tr_data['X_data'] = np.array(data[tr_data_X_name])
            tr_data['y_data'] = np.array(data[tr_data_y_name])

            training_sample_count = tr_data['X_data'].shape[0]

            # Initialize model

            model = make_custom_model_cnn_2D()

            # Fit_generator 

            training_sequence_generator = generate_sequences(mini_batch_size,tr_data)


            # Train model with fit_generator

            model.fit_generator(generator=training_sequence_generator,steps_per_epoch=(training_sample_count/(mini_batch_size)+1),epochs=10,verbose=1)    ### step_per_epoch is the number of batches
                                                          ### generator= is the function that makes batches
                                                          ### workers= is the number of multiprocessing  workers=3
            return model        


In [7]:
# from keras.models import load_model

# model = load_model('2018-02-14_model_saved_LRPh5-Copy1.h5')
# # json_file = open("model.json", "r") 
# # loaded_model_json = json_file.read() 
# # json_file.close() 
# # loaded_model = model_from_json(loaded_model_json)

# # loaded_model.load_weights("model.h5") 
# # print("Loaded model from disk")

model = train_model()

this is ith iter :  0
Epoch 1/10
3/3 [==============================] - 2s 699ms/step - loss: 7.9712 - acc: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 99ms/step - loss: 7.9712 - acc: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 102ms/step - loss: 7.9712 - acc: 0.5000
Epoch 4/10
3/3 [==============================] - 0s 102ms/step - loss: 7.9712 - acc: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 104ms/step - loss: 7.9712 - acc: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 88ms/step - loss: 7.9712 - acc: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 8/10
3/3 [==============================] - 0s 106ms/step - loss: 7.9712 - acc: 0.5000
Epoch 9/10
3/3 [==============================] - 0s 110ms/step - loss: 7.9712 - acc: 0.5000
Epoch 10/10
3/3 [==============================] - 0s 113ms/step - loss: 7.9712 - acc: 0.5000


## Analysis

Next, we will set up a list of analysis methods by preparing tuples containing the methods' string identifiers used by `innvestigate.analyzer.create_analyzer(...)`, some optional parameters, a post processing choice for visualizing the computed analysis and a title for the figure to render. Analyzers can be deactivated by simply commenting the corresponding lines, or added by creating a new tuple as below.

For a full list of methods refer to the dictionary `investigate.analyzer.analyzers`

In [8]:
# Configure analysis methods and properties
methods = [
    # NAME                    OPT.PARAMS                POSTPROC FXN               TITLE

    # Show input
    ("lrp.epsilon",           {"epsilon": 1},           ivis.heatmap,        "LRP-Epsilon"),
]

In [9]:
 with h5py.File(data_file_name, "r") as data:

    for i in range(0,1):  # i is the cross number 59

        print('this is ith iter : ' , i)

        tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
        tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

        tr_data = {}
        tr_data['X_data'] = np.array(data[tr_data_X_name])
        tr_data['y_data'] = np.array(data[tr_data_y_name])
        
test_data = tr_data['X_data']
test_label = tr_data['y_data']
print(test_data.shape)

this is ith iter :  0
(16, 68, 95, 79)


The main loop below will now instantiate the analyzer objects based on the loaded/trained model and the analyzers' parameterizations above.

In [10]:
# Create model without trailing softmax
model_wo_softmax = iutils.keras.graph.model_wo_softmax(model)

# Create analyzers.
analyzers = []
for method in methods:
    analyzer = innvestigate.create_analyzer(method[0],        # analysis method identifier
                                            model_wo_softmax, # model without softmax output
                                            **method[1])      # optional analysis parameters

    # Some analyzers require training.
    analyzer.fit(data, batch_size=256, verbose=1)
    analyzers.append(analyzer)
    

Now we analyze each image with the different analyzers:

In [11]:
%%capture

n = 10
test_images = list(zip(test_data[:n], test_label[:n]))

analysis = np.zeros([len(test_images), len(analyzers), 68, 95, 3])
text = []
R=[]



for i, (x, y) in enumerate(test_images):
    # Add batch axis.
    print(x.shape)
    x = x[None, :, :, :]
    
    # Predict final activations, probabilites, and label.
    presm = model_wo_softmax.predict_on_batch(x)[0]
    prob = model.predict_on_batch(x)[0]
    y_hat = prob.argmax()

    
    # Save prediction info:
    text.append(("%s" % label_to_class_name[int(y)],    # ground truth label
                 "%.2f" % presm.max(),             # pre-softmax logits
                 "%.2f" % prob.max(),              # probabilistic softmax output  
                 "%s" % label_to_class_name[y_hat] # predicted label
                ))

    for aidx, analyzer in enumerate(analyzers):
        # Analyze.
        a = analyzer.analyze(x)
        R.append(a)
        # Apply common postprocessing, e.g., re-ordering the channels for plotting.
        a = mnistutils.postprocess(a)
        # Apply analysis postprocessing, e.g., creating a heatmap.
        a = methods[aidx][2](a)
        # Store the analysis.
        analysis[i, aidx] = a[0]
    print(len(R))
    np.save('social_physical_brain_relevance.npy', R)

Next, we visualize the analysis results:

In [12]:
# print(analysis.shape)
# # Prepare the grid as rectengular list
# grid = [[analysis[i, j] for j in range(analysis.shape[1])]
#         for i in range(analysis.shape[0])]
# # Prepare the labels
# label, presm, prob, pred = zip(*text)
# row_labels_left = [('label: {}'.format(label[i]), 'pred: {}'.format(pred[i])) for i in range(len(label))]
# row_labels_right = [('logit: {}'.format(presm[i]), 'prob: {}'.format(prob[i])) for i in range(len(label))]
# col_labels = [''.join(method[3]) for method in methods]

# # Plot the analysis.
# eutils.plot_image_grid(grid, row_labels_left, row_labels_right, col_labels,
#                        file_name=os.environ.get("PLOTFILENAME", None))